# Capstone Project - The Battle of the Neighborhoods (Week 2)
### Applied Data Science Capstone by IBM/Coursera

## Table of contents
* [Introduction: Business Problem](#introduction)
* [Data](#data)
* [Methodology](#methodology)
* [Analysis](#analysis)
* [Conclusion](#conclusion)



## Introduction: Business Problem <a name="introduction"></a>

In this project we will try to find an optimal location for a shop. Specifically, this report will be targeted to stakeholders interested in opening an **Commerce** in **North York**, Canada.

![Map of Neighbourhoods & Communities in Toronto](https://en.healthnexus.ca/sites/en.healthnexus.ca/files/images/main/wellbeingto2.png)

Since there are lots of different shops in Toronti we will try to detect **the best location in North York**. We are also particularly interested in **areas with not activities in vicinity**. We would also prefer locations **as close to city center as possible**, assuming that first two conditions are met.

We will use our data science powers to generate a few most promissing neighborhoods based on this criteria. Advantages of each area will then be clearly expressed so that best possible final location can be chosen by stakeholders.

## Data <a name="data"></a>

Based on definition of our problem, factors that will influence our decission are:
* number of existing shops in the neighborhood (any type)
* number of and distance to other shops in the neighborhood, if any
* distance of neighborhood from city center

We decided to use regularly spaced grid of locations, centered around city center, to define our neighborhoods.

Following data sources will be needed to extract/generate the required information:
* centers of candidate areas will be generated algorithmically and approximate addresses of centers of those areas will be obtained using **Google Maps API reverse geocoding**
* number of restaurants and their type and location in every neighborhood will be obtained using **Foursquare API**
* coordinate of Toronto center will be obtained using **Google Maps API geocoding** of well known North York location.

### Neighborhood Candidates

Extracting and cleaning data of the official web page.

In [ ]:
url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
extracting_data = requests.get(url).text
wiki_data = BeautifulSoup(extracting_data, 'lxml')

column_names = ['Postalcode','Borough','Neighborhood']
toronto = pd.DataFrame(columns = column_names)

content = wiki_data.find('div', class_='mw-parser-output')
table = content.table.tbody
postcode = 0
borough = 0
neighborhood = 0

for tr in table.find_all('tr'):
    i = 0
    for td in tr.find_all('td'):
        if i == 0:
            postcode = td.text.replace('\n','')
            i = i + 1
        elif i == 1:
            borough = td.text.replace('\n','')
            i = i + 1
        elif i == 2: 
            neighborhood = td.text.replace('\n','').replace(']','')
    toronto = toronto.append({'Postalcode': postcode,'Borough': borough,'Neighborhood': neighborhood},ignore_index=True)

toronto = toronto[toronto.Borough!='Not assigned']
toronto = toronto[toronto.Borough!= 0]
toronto.reset_index(drop = True, inplace = True)
i = 0
for i in range(0,toronto.shape[0]):
    if toronto.iloc[i][2] == 'Not assigned':
        toronto.iloc[i][2] = toronto.iloc[i][1]
        i = i+1
        
df = toronto.groupby(['Postalcode','Borough'])['Neighborhood'].apply(', '.join).reset_index()
df.head()

df.describe()

df = df.dropna()
empty = 'Not assigned'
df = df[(df.Postalcode != empty ) & (df.Borough != empty) & (df.Neighborhood != empty)]

def neighborhood_list(grouped):    
    return ', '.join(sorted(grouped['Neighborhood'].tolist()))
                    
grp = df.groupby(['Postalcode', 'Borough'])
df_2 = grp.apply(neighborhood_list).reset_index(name='Neighborhood')

def get_latilong(postal_code):
    lati_long_coords = None
    while(lati_long_coords is None):
        g = geocoder.arcgis('{}, Toronto, Ontario'.format(postal_code))
        lati_long_coords = g.latlng
    return lati_long_coords
    
get_latilong('M4G')

Now let's create a grid of area candidates, equaly spaced, centered around city center and within ~6km from the center of North York. Our neighborhoods will be defined as circular areas with a radius of 300 meters, so our neighborhood centers will be 600 meters apart.

To accurately calculate distances we need to create our grid of locations in Cartesian 2D coordinate system which allows us to calculate distances in meters (not in latitude/longitude degrees). Then we'll project those coordinates back to latitude/longitude degrees to be shown on Folium map. So let's create functions to convert between WGS84 spherical coordinate system (latitude/longitude degrees) and UTM Cartesian coordinate system (X/Y coordinates in  meters).

In [ ]:
from shapely.geometry import *
import pyproj
import math

def lonlat_to_xy(lon, lat):
    proj_latlon = pyproj.Proj(proj='latlong',datum='WGS84')
    proj_xy = pyproj.Proj(proj="utm", zone=33, datum='WGS84')
    xy = pyproj.transform(proj_latlon, proj_xy, lon, lat)
    return xy[0], xy[1]

def xy_to_lonlat(x, y):
    proj_latlon = pyproj.Proj(proj='latlong',datum='WGS84')
    proj_xy = pyproj.Proj(proj="utm", zone=33, datum='WGS84')
    lonlat = pyproj.transform(proj_xy, proj_latlon, x, y)
    return lonlat[0], lonlat[1]

def calc_xy_distance(x1, y1, x2, y2):
    dx = x2 - x1
    dy = y2 - y1
    return math.sqrt(dx*dx + dy*dy)

print('Coordinate transformation check')
print('-------------------------------')
print('North York center longitude={}, latitude={}'.format(NY[1], NY[0]))
x, y = lonlat_to_xy(NY[1], NY[0])
print('North York center UTM X={}, Y={}'.format(x, y))
lo, la = xy_to_lonlat(x, y)
print('North York longitude={}, latitude={}'.format(lo, la))

Let's create a **hexagonal grid of cells**: we offset every other row, and adjust vertical row spacing so that **every cell center is equally distant from all it's neighbors**.

In [ ]:
NY_x, NY_y = lonlat_to_xy(NY[1], NY[0]) # City center in Cartesian coordinates

k = math.sqrt(3) / 2 # Vertical offset for hexagonal grid cells
x_min = NY_x - 6000
x_step = 600
y_min = NY_y - 6000 - (int(21/k)*k*600 - 12000)/2
y_step = 600 * k 

latitudes = []
longitudes = []
distances_from_center = []
xs = []
ys = []
for i in range(0, int(21/k)):
    y = y_min + i * y_step
    x_offset = 300 if i%2==0 else 0
    for j in range(0, 21):
        x = x_min + j * x_step + x_offset
        distance_from_center = calc_xy_distance(NY_x, NY_y, x, y)
        if (distance_from_center <= 6001):
            lon, lat = xy_to_lonlat(x, y)
            latitudes.append(lat)
            longitudes.append(lon)
            distances_from_center.append(distance_from_center)
            xs.append(x)
            ys.append(y)

print(len(latitudes), 'candidate neighborhood centers generated.')

Let's visualize the data we have so far: city center location and candidate neighborhood centers:

In [ ]:
#!pip install folium

import folium

In [ ]:
map_NY = folium.Map(location=NY, zoom_start=13)
folium.Marker(NY, popup='North York').add_to(map_NY)
for lat, lon in zip(latitudes, longitudes):
    #folium.CircleMarker([lat, lon], radius=2, color='blue', fill=True, fill_color='blue', fill_opacity=1).add_to(map_berlin) 
    folium.Circle([lat, lon], radius=300, color='blue', fill=False).add_to(map_NY)
    #folium.Marker([lat, lon]).add_to(map_berlin)
map_NY

OK, we now have the coordinates of centers of neighborhoods/areas to be evaluated, equally spaced (distance from every point to it's neighbors is exactly the same) and within ~6km from North York. 

Let's now use Google Maps API to get approximate addresses of those locations.

In [ ]:

def get_address(api_key, latitude, longitude, verbose=False):
    try:
        url = 'https://maps.googleapis.com/maps/api/geocode/json?key={}&latlng={},{}'.format(api_key, latitude, longitude)
        response = requests.get(url).json()
        if verbose:
            print('Google Maps API JSON result =>', response)
        results = response['results']
        address = results[0]['formatted_address']
        return address
    except:
        return None

addr = get_address(google_api_key, NY[0], NY[1])
print('Reverse geocoding check')
print('-----------------------')
print('Address of [{}, {}] is: {}'.format(NY[0], NY[1], addr))

In [ ]:
print('Obtaining location addresses: ', end='')
addresses = []
for lat, lon in zip(latitudes, longitudes):
    address = get_address(google_api_key, lat, lon)
    if address is None:
        address = 'NO ADDRESS'
    address = address.replace(', Canada', '') # We don't need country part of address
    addresses.append(address)
    print(' .', end='')
print(' done.')

In [ ]:
addresses[150:170]

Looking good. Let's now place all this into a Pandas dataframe.

In [ ]:
import pandas as pd

df_locations = pd.DataFrame({'Address': addresses,
                             'Latitude': latitudes,
                             'Longitude': longitudes,
                             'X': xs,
                             'Y': ys,
                             'Distance from center': distances_from_center})

df_locations.head(10)

...and let's now save/persist this data into local file.

In [ ]:
df_locations.to_pickle('./locations.pkl')    

### Foursquare
Now that we have our location candidates, let's use Foursquare API to get info on restaurants in each neighborhood.

We're interested in venues in 'store' category. So we will include in out list only venues that have 'store' in category name.

Foursquare credentials are defined in hidden cell bellow.

In [ ]:
# The code was removed by Watson Studio for sharing.

In [ ]:
# Category IDs corresponding to Stores (https://developer.foursquare.com/docs/resources/categories):

food_category = '4d954b0ea243a5684a65b473' # 'Root' category for Convenience Store

store_categories = ['4bf58dd8d48988d103951735', '4bf58dd8d48988d114951735', '5745c2e4498e11e7bccabdbd', '4bf58dd8d48988d1f9941735'
                   '52f2ab2ebcbc57f1066b8b1c', '4bf58dd8d48988d111951735', '4bf58dd8d48988d100951735', '4bf58dd8d48988d163941735']

def is_store(categories, specific_filter=None):
    store_words = ['store', 'park', 'clothes', 'convenience']
    store = False
    specific = False
    for c in categories:
        category_name = c[0].lower()
        category_id = c[1]
        for r in rstore_words:
            if r in category_name:
                store = True
        if 'park' in category_name:
            store = False
        if not(specific_filter is None) and (category_id in specific_filter):
            specific = True
            store = True
    return store, specific

def get_categories(categories):
    return [(cat['name'], cat['id']) for cat in categories]

def format_address(location):
    address = ', '.join(location['formattedAddress'])
    address = address.replace(', Canada', '')
    address = address.replace(', Canada', '')
    return address

def get_venues_near_location(lat, lon, category, CLIENT_ID, CLIENT_SECRET, radius=500, limit=100):
    version = '20180724'
    url = 'https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&v={}&ll={},{}&categoryId={}&radius={}&limit={}'.format(
        CLIENT_ID, CLIENT_SECRET, VERSION, lat, lon, category, radius, limit)
    try:
        results = requests.get(url).json()['response']['groups'][0]['items']
        venues = [(item['venue']['id'],
                   item['venue']['name'],
                   get_categories(item['venue']['categories']),
                   (item['venue']['location']['lat'], item['venue']['location']['lng']),
                   format_address(item['venue']['location']),
                   item['venue']['location']['distance']) for item in results]        
    except:
        venues = []
    return venues

Looking good. So now we have all the shopsin area within few kilometers from North York. We also know which restaurants exactly are in vicinity of every neighborhood candidate center.

This concludes the data gathering phase - we're now ready to use this data for analysis to produce the report on optimal locations for a new Italian restaurant!

## Methodology <a name="methodology"></a>

In this project we will direct our efforts on detecting areas of North York that have low store density. We will limit our analysis to area ~6km around city center.

In first step we have collected the required **data: location and type (category) of every store within 6km from North York center**. We have also **identified different stores** (according to Foursquare categorization).

Second step in our analysis will be calculation and exploration of '**store density**' across different areas of Toronto - we will use **heatmaps** to identify a few promising areas close to center with low number of restaurants in general and focus our attention on those areas.

In third and final step we will focus on most promising areas and within those create **clusters of locations that meet some basic requirements** established in discussion with stakeholders: we will take into consideration locations with **no more than two store in radius of 500 meters**. We will present map of all such locations but also create clusters (using **k-means clustering**) of those locations to identify general zones / neighborhoods / addresses which should be a starting point for final 'street level' exploration and search for optimal venue location by stakeholders.

## Analysis <a name="analysis"></a>

Let's perform some basic explanatory data analysis and derive some additional info from our raw data. First let's count the **number of likes in the venue** after that is necessary create new categories for k-means analysis:

In [ ]:
venues = results['response']['groups'][0]['items']
nearby_venues = json_normalize(venues)
filtered_columns = ['venue.name', 'venue.id', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

nearby_venues

#column formatting
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues

Then is necessary creating a list of categorie to remove from our dataframe because they are not store

In [ ]:
removal_list = ['Seafood Restaurant', 'Steakhouse', 'Café', 'Ramen Restaurant', 'Movie Theater', 'Creperie', 'Sushi Restaurant', 'Korean Restaurant', 'Theater', 'Arts & Crafts Store',
       'Japanese Restaurant', 'Bakery', 'Juice Bar', 'Sports Bar', 'Pizza Place', 'Restaurant', 'Fast Food Restaurant', 'Fried Chicken Joint', 'Pool',
       'Dumpling Restaurant', 'Lounge', 'Sandwich Place','Gym', 'American Restaurant', 'Burger Joint', 'Discount Store', 'Middle Eastern Restaurant',
       'Chinese Restaurant', 'Bar', 'Bank', 'Wings Joint','Italian Restaurant', 'Vietnamese Restaurant', 'Karaoke Bar', 'Hotel', 'Noodle House', 'Pub', 'Udon Restaurant']

#getting a clear dataframe of just store
nearby_venues2 = nearby_venues2[~nearby_venues2['categories'].isin(removal_list)]
nearby_venues2

OK, now let's continue with the analysis. Here is possible to see how classify the data about the Like with API of Foursquare

In [ ]:
to_venues['total likes'] = like_list
to_venues.head()

In [ ]:
import matplotlib.pyplot as plt
to_venues['total likes'].hist(bins=4)
plt.show()

In [ ]:
to_venues

OK, let's create our new categories and create a function to apply those to our existing data and apply K-means.

In [ ]:
Groceries = ['Grocery Store', 'Supermarket', 'Bar', 'Dive Bar', 'Sports Bar']
other = ['Shopping Mall', 'Electronics Store', 'Pet Store']
Food_store = ['Coffee Shop', 'Bubble Tea Shop','Ice Cream Shop', 'Dessert Shop']
Plaza = ['Plaza']
Pharmacy = ['Pharmacy']

def conditions2(s):
    if s['categories'] in Groceries:
        return 'Groceries'
    if s['categories'] in other:
        return 'other'
    if s['categories'] in Food_store:
        return 'Food_store'
    if s['categories'] in Plaza:
        return 'Plaza'
    if s['categories'] in Pharmacy:
        return 'Pharmacy'

to_venues['categories_new']=to_venues.apply(conditions2, axis=1)

In [ ]:
cluster_df = to_onehot.drop('Name', axis=1)

k_clusters = 4

# run k-means clustering
kmeans = KMeans(n_clusters=k_clusters, random_state=0).fit(cluster_df)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

In [ ]:
to_venues['label'] = kmeans.labels_
to_venues.head()

In [ ]:
map_clusters = folium.Map(location=[43.77, -79.41], zoom_start=14)

# set color scheme for the clusters
x = np.arange(k_clusters)
ys = [i+x+(i*x)**2 for i in range(k_clusters)]
colors_array = cm.rainbow(np.linspace(0, 100, len(ys)*80))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(to_venues['lat'], to_venues['lng'], to_venues['name'], to_venues['label']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.9).add_to(map_clusters)
       
map_clusters

## Conclusion <a name="conclusion"></a>

Our analysis shows that although there is a lot of number of store in North of York. Highest concentration of store was detected in one of the principal avenue of the city, so we focused our attention this area, also becaus eir near to big place like the Cementery and a lot of parks. 

After directing our attention to this more narrow area of interest we first created a dense grid of location candidates (spaced 100m appart); those locations were then filtered so that those with more than two store in radius of 500m.

Those location candidates were then clustered to create zones of interest which contain greatest number of location candidates. Addresses of centers of those zones were also generated using reverse geocoding to be used as markers/starting points for more detailed local analysis based on other factors.
